In [ ]:
# Advanced Cinnamon Price Forecasting to Excel
# Multiple forecasting modes with comprehensive Excel reporting

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import load_model
import warnings
import pickle
import os
from datetime import datetime
import json
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import LineChart, Reference
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

# Configuration
SEQUENCE_LENGTH = 12
MODEL_DIR = 'cinnamon_models/Quillings'
DATA_PATH = 'C:/VERGER/Spice_Price_Prediction/Cinnamon/Datasets/Cinnamon_Dataset_New_0001_Filled.csv'
OUTPUT_DIR = 'forecast_exports'

# Global variables for preprocessors
scaler_features = None
scaler_target = None
label_encoders = {}
model_config = {}

# Create output directory
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")

print("🌿 Advanced Cinnamon Price Forecasting System")
print("=" * 60)

def list_available_models(model_dir=MODEL_DIR):
    """List all available saved models"""
    if not os.path.exists(model_dir):
        print(f"❌ Model directory not found: {model_dir}")
        return []
    
    model_folders = []
    for item in os.listdir(model_dir):
        item_path = os.path.join(model_dir, item)
        if os.path.isdir(item_path):
            # Check if it contains required model files
            required_files = ['lstm_model.keras', 'scalers.pkl', 'label_encoders.pkl', 'model_config.json']
            if all(os.path.exists(os.path.join(item_path, f)) for f in required_files):
                model_folders.append(item)
    
    return sorted(model_folders, reverse=True)  # Most recent first

def load_saved_model(model_path):
    """Load a previously saved model and preprocessors"""
    global scaler_features, scaler_target, label_encoders, model_config
    
    print(f"📂 Loading model from: {model_path}")
    
    try:
        # Load the Keras model
        keras_model_path = os.path.join(model_path, "lstm_model.keras")
        model = load_model(keras_model_path)
        print(f"✅ Keras model loaded")
        
        # Load scalers
        scalers_path = os.path.join(model_path, "scalers.pkl")
        with open(scalers_path, 'rb') as f:
            scalers = pickle.load(f)
        scaler_features = scalers['scaler_features']
        scaler_target = scalers['scaler_target']
        print(f"✅ Scalers loaded")
        
        # Load label encoders
        encoders_path = os.path.join(model_path, "label_encoders.pkl")
        with open(encoders_path, 'rb') as f:
            label_encoders = pickle.load(f)
        print(f"✅ Label encoders loaded")
        
        # Load configuration
        config_path = os.path.join(model_path, "model_config.json")
        with open(config_path, 'r') as f:
            model_config = json.load(f)
        
        print(f"🎉 Model successfully loaded!")
        print(f"📊 Performance: MAE={model_config['training_info']['mae']:.2f}, "
              f"RMSE={model_config['training_info']['rmse']:.2f}, "
              f"R²={model_config['training_info']['r2']:.4f}")
        
        return model, model_config
        
    except Exception as e:
        print(f"❌ Error loading model: {str(e)}")
        return None, None

def load_and_prepare_data(data_path):
    """Load and prepare the cinnamon price dataset"""
    print(f"📊 Loading data from {data_path}...")
    df = pd.read_csv(data_path)
    print(f"Initial data shape: {df.shape}")

    # Convert Month to datetime
    df['Month'] = pd.to_datetime(df['Month'])

    # Handle missing values in Regional_Price
    missing_before = df['Regional_Price'].isna().sum()
    df.loc[df['Is_Active_Region'] == 0, 'Regional_Price'] = df.loc[df['Is_Active_Region'] == 0, 'National_Price']
    missing_after = df['Regional_Price'].isna().sum()
    print(f"Missing Regional_Price values: {missing_before} -> {missing_after}")

    # Encode categorical variables using loaded encoders
    for col in ['Grade', 'Region']:
        if col in label_encoders:
            df[f'{col}_encoded'] = label_encoders[col].transform(df[col])
        else:
            print(f"⚠️ Warning: No encoder found for {col}")

    # Create additional time-based features
    df['Year'] = df['Month'].dt.year
    df['Month_num'] = df['Month'].dt.month
    df['Quarter'] = df['Month'].dt.quarter

    print("Creating lag and rolling features...")

    # Create lag features for key variables
    df = df.sort_values(['Grade', 'Region', 'Month'])
    lag_columns = ['Regional_Price', 'National_Price', 'Temperature', 'Rainfall']
    for col in lag_columns:
        if col in df.columns:
            for lag in [1, 3, 6, 12]:
                df[f'{col}_lag_{lag}'] = df.groupby(['Grade', 'Region'])[col].shift(lag)

    # Create rolling averages
    for col in ['Regional_Price', 'Temperature', 'Rainfall']:
        if col in df.columns:
            for window in [3, 6, 12]:
                df[f'{col}_rolling_{window}'] = df.groupby(['Grade', 'Region'])[col].transform(
                    lambda x: x.rolling(window).mean()
                )

    print(f"Final data shape after feature engineering: {df.shape}")
    return df

# Define feature columns (same as training)
TRAIN_FEATURE_COLS = [
    'Grade_encoded', 'Region_encoded', 'Is_Active_Region',
    'National_Price', 'Seasonal_Impact', 'Local_Production_Volume',
    'Local_Export_Volume', 'Global_Production_Volume', 'Global_Consumption_Volume',
    'Temperature', 'Rainfall', 'Exchange_Rate', 'Inflation_Rate', 'Fuel_Price',
    'Year', 'Month_num', 'Quarter',
    'Regional_Price_lag_1', 'Regional_Price_lag_3', 'Regional_Price_lag_6', 'Regional_Price_lag_12',
    'National_Price_lag_1', 'National_Price_lag_3', 'National_Price_lag_6', 'National_Price_lag_12',
    'Temperature_lag_1', 'Temperature_lag_3', 'Temperature_lag_6', 'Temperature_lag_12',
    'Rainfall_lag_1', 'Rainfall_lag_3', 'Rainfall_lag_6', 'Rainfall_lag_12',
    'Regional_Price_rolling_3', 'Regional_Price_rolling_6', 'Regional_Price_rolling_12',
    'Temperature_rolling_3', 'Temperature_rolling_6', 'Temperature_rolling_12',
    'Rainfall_rolling_3', 'Rainfall_rolling_6', 'Rainfall_rolling_12'
]

def forecast_prices(model, df, grade, region, months_ahead=12):
    """Generate price forecasts for specified grade and region"""
    subset = df[(df['Grade'] == grade) & (df['Region'] == region)].sort_values('Month')
    
    if len(subset) == 0:
        return None, None, None
    
    last_row = subset.iloc[-1]
    last_date = last_row['Month']
    last_price = last_row['Regional_Price']

    future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1),
                                 periods=months_ahead, freq='MS')
    
    # Generate future rows with realistic seasonal patterns
    future_rows = []
    for future_date in future_dates:
        row = last_row.copy()
        row['Month'] = future_date
        row['Year'] = future_date.year
        row['Month_num'] = future_date.month
        row['Quarter'] = future_date.quarter
        
        # Add seasonal patterns and random variations
        row['Temperature'] = last_row['Temperature'] + 2 * np.sin(2*np.pi*(future_date.month-1)/12) + np.random.normal(0,0.5)
        row['Rainfall'] = max(0, last_row['Rainfall'] + 20 * np.sin(2*np.pi*(future_date.month-1)/12) + np.random.normal(0,10))
        row['Exchange_Rate'] = last_row['Exchange_Rate'] * (1 + np.random.normal(0.001,0.005))
        row['Inflation_Rate'] = last_row['Inflation_Rate'] + np.random.normal(0,0.1)
        row['Fuel_Price'] = last_row['Fuel_Price'] * (1 + np.random.normal(0.002,0.02))
        future_rows.append(row)

    future_df = pd.DataFrame(future_rows)
    extended_df = pd.concat([subset, future_df], ignore_index=True).sort_values('Month')

    # Recreate lag and rolling features for extended data
    for col in ['Regional_Price','National_Price','Temperature','Rainfall']:
        for lag in [1,3,6,12]:
            extended_df[f'{col}_lag_{lag}'] = extended_df.groupby(['Grade','Region'])[col].shift(lag)
        for window in [3,6,12]:
            extended_df[f'{col}_rolling_{window}'] = extended_df.groupby(['Grade','Region'])[col].transform(
                lambda x: x.rolling(window).mean()
            )

    # Select exactly the features used during training
    feature_cols = [c for c in TRAIN_FEATURE_COLS if c in extended_df.columns]

    forecasts = []
    historical_data = extended_df[extended_df['Month'] <= last_date]

    for i in range(months_ahead):
        # Get the sequence needed for prediction
        current_data = extended_df.iloc[len(historical_data)-SEQUENCE_LENGTH+i : len(historical_data)+i]
        
        if len(current_data) < SEQUENCE_LENGTH:
            # Pad with last known data if needed
            padding_needed = SEQUENCE_LENGTH - len(current_data)
            last_known = historical_data.iloc[-1:].copy()
            padding_data = pd.concat([last_known]*padding_needed, ignore_index=True)
            current_data = pd.concat([padding_data, current_data], ignore_index=True).iloc[-SEQUENCE_LENGTH:]

        # Prepare sequence for model
        sequence = current_data[feature_cols].fillna(method='ffill').fillna(method='bfill').values
        sequence_flat = sequence.reshape(-1, sequence.shape[-1])
        sequence_scaled_flat = scaler_features.transform(sequence_flat)
        sequence_scaled = sequence_scaled_flat.reshape(sequence.shape)

        # Make prediction
        next_pred = model.predict(sequence_scaled.reshape(1, SEQUENCE_LENGTH, -1), verbose=0)
        next_pred_unscaled = scaler_target.inverse_transform(next_pred)[0][0]
        forecasts.append(next_pred_unscaled)

        # Update the extended dataframe with the new prediction
        future_idx = len(historical_data)+i
        extended_df.iloc[future_idx, extended_df.columns.get_loc('Regional_Price')] = next_pred_unscaled
        extended_df.iloc[future_idx, extended_df.columns.get_loc('National_Price')] = next_pred_unscaled

    return forecasts, future_dates, last_price

def calculate_forecast_metrics(forecasts, last_price):
    """Calculate forecast metrics"""
    if not forecasts:
        return {}
    
    avg_forecast = np.mean(forecasts)
    min_forecast = np.min(forecasts)
    max_forecast = np.max(forecasts)
    std_forecast = np.std(forecasts)
    volatility = (std_forecast / avg_forecast) * 100
    
    # Calculate trend
    trend_pct = ((forecasts[-1] - forecasts[0]) / forecasts[0]) * 100 if len(forecasts) > 1 else 0
    trend_direction = "Increasing" if trend_pct > 0 else "Decreasing" if trend_pct < 0 else "Stable"
    
    # Risk assessment
    if volatility > 15:
        risk_level = "High"
    elif volatility > 8:
        risk_level = "Medium"
    else:
        risk_level = "Low"
    
    return {
        'last_price': last_price,
        'avg_forecast': avg_forecast,
        'min_forecast': min_forecast,
        'max_forecast': max_forecast,
        'volatility': volatility,
        'trend_pct': trend_pct,
        'trend_direction': trend_direction,
        'risk_level': risk_level
    }

# MODE 1: Regional All Grades Forecast
def generate_regional_forecast(model, df, region, months_ahead=12):
    """Generate forecasts for all grades in a specific region"""
    
    available_grades = sorted(df[df['Region'] == region]['Grade'].unique())
    
    if not available_grades:
        print(f"❌ No grades found for region: {region}")
        return None
    
    print(f"🔮 Mode 1: Generating forecasts for {len(available_grades)} grades in {region}...")
    
    batch_results = {}
    
    for i, grade in enumerate(available_grades, 1):
        print(f"   📊 Processing {grade} ({i}/{len(available_grades)})...")
        
        try:
            forecasts, future_dates, last_price = forecast_prices(model, df, grade, region, months_ahead)
            
            if forecasts is not None:
                metrics = calculate_forecast_metrics(forecasts, last_price)
                
                batch_results[grade] = {
                    'forecasts': forecasts,
                    'future_dates': future_dates,
                    **metrics
                }
                print(f"      ✅ Success - Avg: LKR {metrics['avg_forecast']:,.0f}, Trend: {metrics['trend_pct']:+.1f}%")
            else:
                print(f"      ❌ Failed to generate forecast")
                
        except Exception as e:
            print(f"      ❌ Error: {str(e)}")
    
    print(f"✅ Regional forecast completed! {len(batch_results)}/{len(available_grades)} successful.")
    return batch_results

# MODE 2: Grade Regional Deviation Analysis
def generate_grade_regional_analysis(model, df, grade, months_ahead=12):
    """Generate regional price deviation analysis for a specific grade"""
    
    available_regions = sorted(df[df['Grade'] == grade]['Region'].unique())
    
    if not available_regions:
        print(f"❌ No regions found for grade: {grade}")
        return None
    
    print(f"🔮 Mode 2: Analyzing {grade} across {len(available_regions)} regions...")
    
    regional_results = {}
    
    for i, region in enumerate(available_regions, 1):
        print(f"   🗺️ Processing {region} ({i}/{len(available_regions)})...")
        
        try:
            forecasts, future_dates, last_price = forecast_prices(model, df, grade, region, months_ahead)
            
            if forecasts is not None:
                metrics = calculate_forecast_metrics(forecasts, last_price)
                
                regional_results[region] = {
                    'forecasts': forecasts,
                    'future_dates': future_dates,
                    **metrics
                }
                print(f"✅ Success - Avg: LKR {metrics['avg_forecast']:,.0f}, Trend: {metrics['trend_pct']:+.1f}%")
            else:
                print(f"❌ Failed to generate forecast")
                
        except Exception as e:
            print(f"❌ Error: {str(e)}")
    
    print(f"✅ Grade regional analysis completed! {len(regional_results)}/{len(available_regions)} successful.")
    
    # Calculate regional deviations
    if len(regional_results) > 1:
        avg_prices = [data['avg_forecast'] for data in regional_results.values()]
        national_avg = np.mean(avg_prices)
        
        for region, data in regional_results.items():
            deviation = data['avg_forecast'] - national_avg
            deviation_pct = (deviation / national_avg) * 100
            regional_results[region]['price_deviation'] = deviation
            regional_results[region]['deviation_pct'] = deviation_pct
            regional_results[region]['national_avg'] = national_avg
    
    return regional_results

# MODE 3: Month-by-Month Forecasts for All Combinations
def generate_comprehensive_monthly_report(model, df, months_ahead=12):
    """Generate month-by-month forecasts for all grade-region combinations"""
    
    # Get all unique combinations
    combinations = []
    for _, row in df.groupby(['Grade', 'Region']).size().reset_index().iterrows():
        combinations.append((row['Grade'], row['Region']))
    
    print(f"🔮 Mode 3: Generating month-by-month forecasts for {len(combinations)} grade-region combinations...")
    
    comprehensive_results = {}
    successful_forecasts = 0
    
    for i, (grade, region) in enumerate(combinations, 1):
        print(f"   📊 Processing {grade} - {region} ({i}/{len(combinations)})...")
        
        try:
            forecasts, future_dates, last_price = forecast_prices(model, df, grade, region, months_ahead)
            
            if forecasts is not None:
                metrics = calculate_forecast_metrics(forecasts, last_price)
                
                key = f"{grade}_{region}"
                comprehensive_results[key] = {
                    'grade': grade,
                    'region': region,
                    'forecasts': forecasts,
                    'future_dates': future_dates,
                    'last_price': last_price,
                    **metrics
                }
                successful_forecasts += 1
                if successful_forecasts % 10 == 0:
                    print(f"      📈 Progress: {successful_forecasts} forecasts completed...")
            else:
                print(f"      ❌ Failed")
                
        except Exception as e:
            print(f"      ❌ Error: {str(e)}")
    
    print(f"✅ Comprehensive report completed! {successful_forecasts}/{len(combinations)} successful.")
    return comprehensive_results

def create_excel_report_mode1(batch_results, region, months_ahead, output_dir=OUTPUT_DIR):
    """Create Excel report for Mode 1 - Regional All Grades"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Mode1_Regional_Forecast_{region}_{months_ahead}months_{timestamp}.xlsx"
    filepath = os.path.join(output_dir, filename)
    
    print(f"📝 Creating Mode 1 Excel report: {filename}")
    
    # Create workbook
    wb = Workbook()
    wb.remove(wb.active)
    
    # Define styling
    header_font = Font(bold=True, color='FFFFFF')
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                   top=Side(style='thin'), bottom=Side(style='thin'))
    center_align = Alignment(horizontal='center', vertical='center')
    
    # Sheet 1: Summary Dashboard
    ws_summary = wb.create_sheet("Regional Summary")
    
    # Headers
    ws_summary['A1'] = f"🌿 Regional Forecast Summary - {region}"
    ws_summary['A1'].font = Font(bold=True, size=16)
    ws_summary['A2'] = f"📅 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    ws_summary['A3'] = f"📊 Forecast Period: {months_ahead} months"
    ws_summary['A4'] = f"🏆 Model R²: {model_config.get('training_info', {}).get('r2', 'N/A'):.4f}"
    
    # Summary table
    headers = ['Grade', 'Last Price (LKR)', 'Avg Forecast (LKR)', 'Min Forecast (LKR)', 
               'Max Forecast (LKR)', 'Volatility (%)', 'Trend (%)', 'Trend Direction', 'Risk Level']
    
    for col, header in enumerate(headers, 1):
        cell = ws_summary.cell(row=6, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    # Fill data
    row = 7
    for grade, data in batch_results.items():
        ws_summary[f'A{row}'] = grade
        ws_summary[f'B{row}'] = round(data['last_price'], 2)
        ws_summary[f'C{row}'] = round(data['avg_forecast'], 2)
        ws_summary[f'D{row}'] = round(data['min_forecast'], 2)
        ws_summary[f'E{row}'] = round(data['max_forecast'], 2)
        ws_summary[f'F{row}'] = round(data['volatility'], 2)
        ws_summary[f'G{row}'] = round(data['trend_pct'], 2)
        ws_summary[f'H{row}'] = data['trend_direction']
        ws_summary[f'I{row}'] = data['risk_level']
        
        # Apply styling and color coding
        for col in range(1, 10):
            cell = ws_summary.cell(row=row, column=col)
            cell.border = border
            cell.alignment = center_align
            
            # Color code risk levels
            if col == 9 and data['risk_level'] == 'High':
                cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
            elif col == 9 and data['risk_level'] == 'Low':
                cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
            elif col == 7:  # Trend column
                if data['trend_pct'] > 0:
                    cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
                elif data['trend_pct'] < 0:
                    cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
        
        row += 1
    
    # Auto-adjust column widths
    for column in ws_summary.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws_summary.column_dimensions[column_letter].width = adjusted_width
    
    # Sheet 2: Detailed Monthly Forecasts
    ws_detailed = wb.create_sheet("Monthly Forecasts")
    
    ws_detailed['A1'] = f"📅 Monthly Forecasts - {region}"
    ws_detailed['A1'].font = Font(bold=True, size=14)
    
    # Create detailed table
    detail_headers = ['Month', 'Date'] + list(batch_results.keys())
    for col, header in enumerate(detail_headers, 1):
        cell = ws_detailed.cell(row=3, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    # Fill monthly data
    first_grade = list(batch_results.keys())[0]
    future_dates = batch_results[first_grade]['future_dates']
    
    for month_idx, date in enumerate(future_dates):
        row_num = month_idx + 4
        ws_detailed[f'A{row_num}'] = f"Month {month_idx + 1}"
        ws_detailed[f'B{row_num}'] = date.strftime('%Y-%m')
        
        for col_idx, grade in enumerate(batch_results.keys(), 3):
            forecast_value = batch_results[grade]['forecasts'][month_idx]
            cell = ws_detailed.cell(row=row_num, column=col_idx, value=round(forecast_value, 2))
            cell.border = border
            cell.alignment = center_align
    
    # Auto-adjust column widths
    for column in ws_detailed.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 15)
        ws_detailed.column_dimensions[column_letter].width = adjusted_width
    
    try:
        wb.save(filepath)
        print(f"✅ Mode 1 Excel report saved successfully: {filename}")
        return filepath
    except Exception as e:
        print(f"❌ Error saving Excel file: {str(e)}")
        return None

def create_excel_report_mode2(regional_results, grade, months_ahead, output_dir=OUTPUT_DIR):
    """Create Excel report for Mode 2 - Grade Regional Deviation"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Mode2_Grade_Regional_{grade}_{months_ahead}months_{timestamp}.xlsx"
    filepath = os.path.join(output_dir, filename)
    
    print(f"📝 Creating Mode 2 Excel report: {filename}")
    
    # Create workbook
    wb = Workbook()
    wb.remove(wb.active)
    
    # Define styling
    header_font = Font(bold=True, color='FFFFFF')
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                   top=Side(style='thin'), bottom=Side(style='thin'))
    center_align = Alignment(horizontal='center', vertical='center')
    
    # Sheet 1: Regional Deviation Summary
    ws_summary = wb.create_sheet("Regional Deviation")
    
    ws_summary['A1'] = f"🗺️ Regional Price Deviation Analysis - {grade}"
    ws_summary['A1'].font = Font(bold=True, size=16)
    ws_summary['A2'] = f"📅 Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    ws_summary['A3'] = f"📊 Forecast Period: {months_ahead} months"
    
    # Check if deviation data exists
    has_deviation_data = any('price_deviation' in data for data in regional_results.values())
    
    if has_deviation_data:
        headers = ['Region', 'Last Price (LKR)', 'Avg Forecast (LKR)', 'National Avg (LKR)',
                   'Price Deviation (LKR)', 'Deviation %', 'Volatility (%)', 'Trend (%)', 'Risk Level']
        
        # Add national average info
        first_region_data = list(regional_results.values())[0]
        national_avg = first_region_data.get('national_avg', 0)
        ws_summary['A4'] = f"📊 National Average Forecast: LKR {national_avg:,.2f}"
    else:
        headers = ['Region', 'Last Price (LKR)', 'Avg Forecast (LKR)', 
                   'Volatility (%)', 'Trend (%)', 'Risk Level']
    
    # Create headers
    for col, header in enumerate(headers, 1):
        cell = ws_summary.cell(row=6, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    # Fill data
    row = 7
    for region, data in regional_results.items():
        col_idx = 1
        ws_summary.cell(row=row, column=col_idx, value=region)
        col_idx += 1
        
        ws_summary.cell(row=row, column=col_idx, value=round(data['last_price'], 2))
        col_idx += 1
        
        ws_summary.cell(row=row, column=col_idx, value=round(data['avg_forecast'], 2))
        col_idx += 1
        
        if has_deviation_data:
            ws_summary.cell(row=row, column=col_idx, value=round(data.get('national_avg', 0), 2))
            col_idx += 1
            
            ws_summary.cell(row=row, column=col_idx, value=round(data.get('price_deviation', 0), 2))
            col_idx += 1
            
            ws_summary.cell(row=row, column=col_idx, value=round(data.get('deviation_pct', 0), 2))
            col_idx += 1
        
        ws_summary.cell(row=row, column=col_idx, value=round(data['volatility'], 2))
        col_idx += 1
        
        ws_summary.cell(row=row, column=col_idx, value=round(data['trend_pct'], 2))
        col_idx += 1
        
        ws_summary.cell(row=row, column=col_idx, value=data['risk_level'])
        
        # Apply styling
        for col in range(1, len(headers) + 1):
            cell = ws_summary.cell(row=row, column=col)
            cell.border = border
            cell.alignment = center_align
            
            # Color code deviations if available
            if has_deviation_data and col == 6:  # Deviation % column
                deviation_pct = data.get('deviation_pct', 0)
                if deviation_pct > 5:
                    cell.fill = PatternFill(start_color='FFE5E5', end_color='FFE5E5', fill_type='solid')
                elif deviation_pct < -5:
                    cell.fill = PatternFill(start_color='E5F5E5', end_color='E5F5E5', fill_type='solid')
            
            # Color code risk levels
            if col == len(headers):  # Last column (Risk Level)
                if data['risk_level'] == 'High':
                    cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
                elif data['risk_level'] == 'Low':
                    cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
        
        row += 1
    
    # Auto-adjust column widths
    for column in ws_summary.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws_summary.column_dimensions[column_letter].width = adjusted_width
    
    # Sheet 2: Monthly Regional Comparison
    ws_monthly = wb.create_sheet("Monthly Comparison")
    
    ws_monthly['A1'] = f"📅 Monthly Regional Price Comparison - {grade}"
    ws_monthly['A1'].font = Font(bold=True, size=14)
    
    # Create monthly comparison table
    detail_headers = ['Month', 'Date'] + list(regional_results.keys())
    for col, header in enumerate(detail_headers, 1):
        cell = ws_monthly.cell(row=3, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    # Fill monthly data
    first_region = list(regional_results.keys())[0]
    future_dates = regional_results[first_region]['future_dates']
    
    for month_idx, date in enumerate(future_dates):
        row_num = month_idx + 4
        ws_monthly[f'A{row_num}'] = f"Month {month_idx + 1}"
        ws_monthly[f'B{row_num}'] = date.strftime('%Y-%m')
        
        for col_idx, region in enumerate(regional_results.keys(), 3):
            forecast_value = regional_results[region]['forecasts'][month_idx]
            cell = ws_monthly.cell(row=row_num, column=col_idx, value=round(forecast_value, 2))
            cell.border = border
            cell.alignment = center_align
    
    # Auto-adjust column widths
    for column in ws_monthly.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 15)
        ws_monthly.column_dimensions[column_letter].width = adjusted_width
    
    try:
        wb.save(filepath)
        print(f"✅ Mode 2 Excel report saved successfully: {filename}")
        return filepath
    except Exception as e:
        print(f"❌ Error saving Excel file: {str(e)}")
        return None

def create_excel_report_mode3(comprehensive_results, months_ahead, output_dir=OUTPUT_DIR):
    """Create Excel report for Mode 3 - Month-by-Month Comprehensive Report"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"Mode3_Monthly_AllCombinations_{months_ahead}months_{timestamp}.xlsx"
    filepath = os.path.join(output_dir, filename)
    
    print(f"📝 Creating Mode 3 Month-by-Month Excel report: {filename}")
    print(f"   Processing {len(comprehensive_results)} grade-region combinations...")
    
    # Create workbook
    wb = Workbook()
    wb.remove(wb.active)
    
    # Define styling
    header_font = Font(bold=True, color='FFFFFF')
    header_fill = PatternFill(start_color='366092', end_color='366092', fill_type='solid')
    border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                   top=Side(style='thin'), bottom=Side(style='thin'))
    center_align = Alignment(horizontal='center', vertical='center')
    
    # Sheet 1: Month-by-Month Prices for All Combinations
    ws_monthly = wb.create_sheet("Monthly Prices - All Combinations")
    
    ws_monthly['A1'] = "📅 Month-by-Month Price Forecasts - All Grade-Region Combinations"
    ws_monthly['A1'].font = Font(bold=True, size=16)
    ws_monthly['A2'] = f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
    ws_monthly['A3'] = f"Forecast Period: {months_ahead} months"
    ws_monthly['A4'] = f"Total Combinations: {len(comprehensive_results)}"
    ws_monthly['A5'] = f"Model R²: {model_config.get('training_info', {}).get('r2', 'N/A'):.4f}"
    
    # Get the month dates from first result
    if comprehensive_results:
        first_key = list(comprehensive_results.keys())[0]
        future_dates = comprehensive_results[first_key]['future_dates']
        
        # Create headers: Grade | Region | Last Price | Month 1 | Month 2 | ... | Month N
        headers = ['Grade', 'Region', 'Last Price (LKR)']
        for i, date in enumerate(future_dates, 1):
            headers.append(f"Month {i}\n{date.strftime('%Y-%m')}")
        
        # Add headers to sheet
        for col, header in enumerate(headers, 1):
            cell = ws_monthly.cell(row=7, column=col, value=header)
            cell.font = header_font
            cell.fill = header_fill
            cell.alignment = center_align
            cell.border = border
        
        # Fill data for each grade-region combination
        row = 8
        for key, data in sorted(comprehensive_results.items()):
            # Grade and Region
            ws_monthly.cell(row=row, column=1, value=data['grade'])
            ws_monthly.cell(row=row, column=2, value=data['region'])
            ws_monthly.cell(row=row, column=3, value=round(data['last_price'], 2))
            
            # Apply styling to first 3 columns
            for col in range(1, 4):
                cell = ws_monthly.cell(row=row, column=col)
                cell.border = border
                cell.alignment = center_align
            
            # Monthly forecasts
            for month_idx, forecast_value in enumerate(data['forecasts']):
                col_num = month_idx + 4
                cell = ws_monthly.cell(row=row, column=col_num, value=round(forecast_value, 2))
                cell.border = border
                cell.alignment = center_align
                
                # Color code based on change from last price
                change_pct = ((forecast_value - data['last_price']) / data['last_price']) * 100
                if change_pct > 5:
                    cell.fill = PatternFill(start_color='C8E6C9', end_color='C8E6C9', fill_type='solid')
                elif change_pct < -5:
                    cell.fill = PatternFill(start_color='FFCDD2', end_color='FFCDD2', fill_type='solid')
            
            row += 1
        
        # Auto-adjust column widths
        for column in ws_monthly.columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(str(cell.value))
                except:
                    pass
            adjusted_width = min(max_length + 2, 18)
            ws_monthly.column_dimensions[column_letter].width = adjusted_width
    
    # Sheet 2: Summary Statistics
    ws_summary = wb.create_sheet("Summary Statistics")
    
    ws_summary['A1'] = "📊 Forecast Summary Statistics"
    ws_summary['A1'].font = Font(bold=True, size=14)
    
    summary_headers = ['Grade', 'Region', 'Last Price (LKR)', 'Avg Forecast (LKR)', 
                       'Min Forecast (LKR)', 'Max Forecast (LKR)', 'Price Range (LKR)',
                       'Volatility (%)', 'Trend (%)', 'Risk Level']
    
    for col, header in enumerate(summary_headers, 1):
        cell = ws_summary.cell(row=3, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    row = 4
    for key, data in sorted(comprehensive_results.items()):
        ws_summary.cell(row=row, column=1, value=data['grade'])
        ws_summary.cell(row=row, column=2, value=data['region'])
        ws_summary.cell(row=row, column=3, value=round(data['last_price'], 2))
        ws_summary.cell(row=row, column=4, value=round(data['avg_forecast'], 2))
        ws_summary.cell(row=row, column=5, value=round(data['min_forecast'], 2))
        ws_summary.cell(row=row, column=6, value=round(data['max_forecast'], 2))
        ws_summary.cell(row=row, column=7, value=round(data['max_forecast'] - data['min_forecast'], 2))
        ws_summary.cell(row=row, column=8, value=round(data['volatility'], 2))
        ws_summary.cell(row=row, column=9, value=round(data['trend_pct'], 2))
        ws_summary.cell(row=row, column=10, value=data['risk_level'])
        
        for col in range(1, 11):
            cell = ws_summary.cell(row=row, column=col)
            cell.border = border
            cell.alignment = center_align
            
            if col == 10:  # Risk Level
                if data['risk_level'] == 'High':
                    cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
                elif data['risk_level'] == 'Low':
                    cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
            elif col == 9:  # Trend
                if data['trend_pct'] > 5:
                    cell.fill = PatternFill(start_color='E8F5E8', end_color='E8F5E8', fill_type='solid')
                elif data['trend_pct'] < -5:
                    cell.fill = PatternFill(start_color='FFEBEE', end_color='FFEBEE', fill_type='solid')
        
        row += 1
    
    for column in ws_summary.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 20)
        ws_summary.column_dimensions[column_letter].width = adjusted_width
    
    # Sheet 3: Grade-wise Breakdown
    ws_grade = wb.create_sheet("Grade-wise Analysis")
    
    ws_grade['A1'] = "🏷️ Grade-wise Price Analysis"
    ws_grade['A1'].font = Font(bold=True, size=14)
    
    # Group by grade
    grade_data = {}
    for key, data in comprehensive_results.items():
        grade = data['grade']
        if grade not in grade_data:
            grade_data[grade] = {'prices': [], 'regions': []}
        grade_data[grade]['prices'].append(data['avg_forecast'])
        grade_data[grade]['regions'].append(data['region'])
    
    grade_headers = ['Grade', 'Avg Price (LKR)', 'Min Price (LKR)', 
                     'Max Price (LKR)', 'Price Range (LKR)', 'Regions Count']
    
    for col, header in enumerate(grade_headers, 1):
        cell = ws_grade.cell(row=3, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    row = 4
    for grade in sorted(grade_data.keys()):
        prices = grade_data[grade]['prices']
        ws_grade.cell(row=row, column=1, value=grade)
        ws_grade.cell(row=row, column=2, value=round(np.mean(prices), 2))
        ws_grade.cell(row=row, column=3, value=round(np.min(prices), 2))
        ws_grade.cell(row=row, column=4, value=round(np.max(prices), 2))
        ws_grade.cell(row=row, column=5, value=round(np.max(prices) - np.min(prices), 2))
        ws_grade.cell(row=row, column=6, value=len(prices))
        
        for col in range(1, 7):
            cell = ws_grade.cell(row=row, column=col)
            cell.border = border
            cell.alignment = center_align
        
        row += 1
    
    # Sheet 4: Region-wise Breakdown
    ws_region = wb.create_sheet("Region-wise Analysis")
    
    ws_region['A1'] = "🗺️ Region-wise Price Analysis"
    ws_region['A1'].font = Font(bold=True, size=14)
    
    # Group by region
    region_data = {}
    for key, data in comprehensive_results.items():
        region = data['region']
        if region not in region_data:
            region_data[region] = {'prices': [], 'grades': []}
        region_data[region]['prices'].append(data['avg_forecast'])
        region_data[region]['grades'].append(data['grade'])
    
    region_headers = ['Region', 'Avg Price (LKR)', 'Min Price (LKR)', 
                      'Max Price (LKR)', 'Price Range (LKR)', 'Grades Count']
    
    for col, header in enumerate(region_headers, 1):
        cell = ws_region.cell(row=3, column=col, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = center_align
        cell.border = border
    
    row = 4
    for region in sorted(region_data.keys()):
        prices = region_data[region]['prices']
        ws_region.cell(row=row, column=1, value=region)
        ws_region.cell(row=row, column=2, value=round(np.mean(prices), 2))
        ws_region.cell(row=row, column=3, value=round(np.min(prices), 2))
        ws_region.cell(row=row, column=4, value=round(np.max(prices), 2))
        ws_region.cell(row=row, column=5, value=round(np.max(prices) - np.min(prices), 2))
        ws_region.cell(row=row, column=6, value=len(prices))
        
        for col in range(1, 7):
            cell = ws_region.cell(row=row, column=col)
            cell.border = border
            cell.alignment = center_align
        
        row += 1
    
    try:
        wb.save(filepath)
        print(f"✅ Mode 3 Month-by-Month Excel report saved successfully: {filename}")
        print(f"   📊 Sheets: Monthly Prices (All Combinations), Summary Statistics, Grade-wise, Region-wise")
        return filepath
    except Exception as e:
        print(f"❌ Error saving Excel file: {str(e)}")
        return None

def main():
    print(f"\n{"="*60}")
    print("🌿 AGRICULTURAL COMMODITY PRICE FORECASTING TOOL")
    
    # List and select model
    available_models = list_available_models()
    
    if not available_models:
        print("❌ No saved models found!")
        return
    
    print("📂 Available Models:")
    for i, model_name in enumerate(available_models, 1):
        print(f"{i}. {model_name}")
    
    try:
        selection = input(f"\n🎯 Select model (1-{len(available_models)}, default=1): ").strip()
        model_idx = (int(selection) - 1) if selection else 0
        
        if 0 <= model_idx < len(available_models):
            selected_model = available_models[model_idx]
            model_path = os.path.join(MODEL_DIR, selected_model)
        else:
            print("❌ Invalid selection!")
            return
    except ValueError:
        print("❌ Invalid input!")
        return
    
    # Load model and data
    model, config = load_saved_model(model_path)
    if model is None:
        return
    
    df = load_and_prepare_data(DATA_PATH)
    
    # Display forecast mode options
    print(f"\n{'='*60}")
    print("📋 FORECAST MODE SELECTION")
    print("=" * 60)
    print("1. 🗺️ Regional Analysis - All grades for a specific region")
    print("2. 🏷️ Grade Analysis - One grade across all regions (Price Deviation)")
    print("3. 📅 Month-by-Month Report - All grade-region combinations with monthly prices")
    print("4. 🚪 Exit")
    
    try:
        mode = input("\n🎯 Select forecast mode (1-4): ").strip()
        
        if mode == '4':
            print("👋 Goodbye!")
            return
        
        # Get forecast period
        months_ahead = int(input("📅 Number of months to forecast (default 6): ") or "6")
        if months_ahead <= 0 or months_ahead > 24:
            print("⚠️ Months should be between 1 and 24. Using default (6).")
            months_ahead = 6
        
        print(f"\n{'='*60}")
        
        if mode == '1':
            # Mode 1: Regional All Grades
            available_regions = sorted(df['Region'].unique())
            print(f"🗺️ MODE 1: REGIONAL ANALYSIS")
            print("=" * 60)
            print(f"Available Regions: {', '.join(available_regions)}")
            
            region = input(f"\n🗺️ Enter region: ").strip()
            if region not in available_regions:
                print("❌ Invalid region!")
                return
            
            print(f"\n{'='*60}")
            results = generate_regional_forecast(model, df, region, months_ahead)
            
            if results:
                excel_path = create_excel_report_mode1(results, region, months_ahead)
                if excel_path:
                    print(f"\n🎉 Regional analysis completed!")
                    print(f"📄 Report saved: {os.path.basename(excel_path)}")
        
        elif mode == '2':
            # Mode 2: Grade Regional Deviation
            available_grades = sorted(df['Grade'].unique())
            print(f"🏷️ MODE 2: GRADE REGIONAL ANALYSIS")
            print("=" * 60)
            print(f"Available Grades: {', '.join(available_grades)}")
            
            grade = input(f"\n🏷️ Enter grade: ").strip()
            if grade not in available_grades:
                print("❌ Invalid grade!")
                return
            
            print(f"\n{'='*60}")
            results = generate_grade_regional_analysis(model, df, grade, months_ahead)
            
            if results:
                excel_path = create_excel_report_mode2(results, grade, months_ahead)
                if excel_path:
                    print(f"\n🎉 Grade regional analysis completed!")
                    print(f"📄 Report saved: {os.path.basename(excel_path)}")
        
        elif mode == '3':
            # Mode 3: Month-by-Month Comprehensive Report
            print(f"📅 MODE 3: MONTH-BY-MONTH COMPREHENSIVE ANALYSIS")
            print("=" * 60)
            print("⚠️ This will generate month-by-month forecasts for ALL grade-region combinations.")
            print("⏱️ This may take several minutes depending on data size.")
            
            confirm = input("\n❓ Continue? (yes/no): ").strip().lower()
            if confirm != 'yes':
                print("❌ Operation cancelled.")
                return
            
            print(f"\n{'='*60}")
            results = generate_comprehensive_monthly_report(model, df, months_ahead)
            
            if results:
                excel_path = create_excel_report_mode3(results, months_ahead)
                if excel_path:
                    print(f"\n🎉 Month-by-month comprehensive analysis completed!")
                    print(f"📄 Report saved: {os.path.basename(excel_path)}")
        
        else:
            print("❌ Invalid mode selection!")
            return
            
    except ValueError as e:
        print(f"❌ Invalid input: {e}")
        return
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return

if __name__ == "__main__":
    main()

🌿 Advanced Cinnamon Price Forecasting System

🌿 AGRICULTURAL COMMODITY PRICE FORECASTING TOOL
📂 Available Models:
1. cinnamon_model_20250929_150355
2. cinnamon_model_20250926_160758
3. cinnamon_model_20250926_153157
4. cinnamon_model_20250925_161931
5. cinnamon_model_20250925_150447
6. cinnamon_model_20250924_114850
7. cinnamon_model_20250923_183211
8. cinnamon_model_20250923_151802
📂 Loading model from: cinnamon_models\cinnamon_model_20250929_150355
✅ Keras model loaded
✅ Scalers loaded
✅ Label encoders loaded
🎉 Model successfully loaded!
📊 Performance: MAE=67.34, RMSE=101.93, R²=0.9859
📊 Loading data from C:/VERGER/Spice_Price_Prediction/Cinnamon/Datasets/Cinnamon_Dataset_New_0001_Filled.csv...
Initial data shape: (2898, 16)
Missing Regional_Price values: 0 -> 0
Creating lag and rolling features...
Final data shape after feature engineering: (2898, 46)

📋 FORECAST MODE SELECTION
1. 🗺️ Regional Analysis - All grades for a specific region
2. 🏷️ Grade Analysis - One grade across all reg